# Get discussion structures

This notebook pretty-prints the structure of discussions to give a sense of the conversation in a forum.

Each reply in the forum is represented by an identifier like `R2___HyxmaXDd0Q`; in general, `{commenter_identifier}___{reply_note_id}`. The commenter identifier comes from the set:


| Identifier | Commenter type |
|------------|----------------|
| A          | Authors        |
| C          | Conference     |
| Ri         | Reviewer i     |
| N          | A**N**onymous      |
| ACi        | Area Chair i   |
| S          | **S**omeone else   |

You may have to change these categories and the way they are determined in the `get_author` function if you decide to use a conference besides ICLR 2019.

This notebook uses the Python package [pptree](https://github.com/clemtoy/pptree).

In [1]:
import openreview
import tqdm

# Change these values according to your needs
INVITATION = 'ICLR.cc/2019/Conference/-/Blind_Submission'
# A client is required for any OpenReview API actions
guest_client = openreview.Client(baseurl='https://api.openreview.net')
LIMIT = 10

In [2]:
class NoteNode(object):
  def __init__(self, note_id, reply_to):
    self.note_id = note_id
    self.reply_to = reply_to
    self.children = []
    
import pptree

def get_author(note):
  sig = note.signatures[0].split("/")[-1]
  if sig == 'Authors':
    return "A"
  elif sig == "Conference":
    return "C"
  elif sig == "(anonymous)":
    return "N"
  elif "Area_Chair" in sig:
    return "AC" + sig[-1]
  elif "AnonReviewer" in sig:
    return "R" + sig[-1]
  elif sig.startswith("~"):
    return "S" # Someone
  else:
    print(sig)
    dsds
    
    
def get_review_discussions(notes):
  ordered_notes = sorted([(note.tcdate, note.id, note.replyto, get_author(note), note) for note in notes])
  original_id_to_node = {}
  root_node = None
  valid_reply_tos = [x[1] for x in ordered_notes]
  for _, note_id, reply_to, author_indicator, note in ordered_notes:
    if reply_to is None:
      original_root_note = note
    new_id = f'{author_indicator}___{note_id}'
    if reply_to is None:
      original_id_to_node[note_id] = pptree.Node(new_id)
      root_node = original_id_to_node[note_id]
    elif reply_to not in valid_reply_tos:
      return None
    else:
      original_id_to_node[note_id] = pptree.Node(new_id, original_id_to_node[reply_to])

  print("-" * 80)
  print(f'Title: {original_root_note.content["title"]}\nPermalink: https://openreview.net/forum?id={original_root_note.forum}')
  pptree.print_tree(root_node)
  print()


In [3]:
for i, forum_note in enumerate(openreview.tools.iterget_notes(
        guest_client, invitation=INVITATION)):
    this_forum_notes = guest_client.get_notes(forum=forum_note.id)
    get_review_discussions(this_forum_notes)
    if i == LIMIT - 1:
      break


--------------------------------------------------------------------------------
Title: Regularized Learning for  Domain Adaptation under Label Shifts
Permalink: https://openreview.net/forum?id=rJl0r3R9KX
               ┌A___BJg59Q0iT7
               ├AC1___HJlQWqQreN
               ├R2___SyxJIBjuhm┐
               │               └A___BkeG9kNh6X
 C___rJl0r3R9KX┤
               ├R1___rkl-PP8d2Q┐
               │               └A___rygX0-Ehp7┐
               │                              └R1___BJe5ulInCQ┐
               │                                              └A___S1xCYXQGyV
               └R3___r1ghyrDc2m┐
                               └A___Hkldv4Aj6X

--------------------------------------------------------------------------------
Title: Towards Robust, Locally Linear Deep Networks
Permalink: https://openreview.net/forum?id=SylCrnCcFX
               ┌A___ByeYOl_OT7
               ├A___SygRbZudpQ
               ├AC1___SJljqe6RyV
               ├N___BJxlHFwdlV
               ├R